# 수치미분 자동미분

수치미분
    수치미분은 함수의 미분값을 근사적으로 계산하는 방법입니다. 가장 일반적인 방법은 차분 방법 (finite difference method)입니다. 차분 방법은 함수의 작은 변화를 이용하여 미분값을 계산합니다.

        전진 차분 (Forward Difference)
        전진 차분은 다음과 같이 계산됩니다:

        [ f'(x) \approx \frac{f(x + h) - f(x)}{h} ]

        여기서 ( h )는 매우 작은 값입니다.

        후진 차분 (Backward Difference)
        후진 차분은 다음과 같습니다:

        [ f'(x) \approx \frac{f(x) - f(x - h)}{h} ]

        중앙 차분 (Central Difference)
        중앙 차분은 전진 차분과 후진 차분의 평균을 사용하여 더 정확한 값을 제공합니다:

        [ f'(x) \approx \frac{f(x + h) - f(x - h)}{2h} ]

In [2]:
def f(x):
    return x**2

def numerical_derivative(f, x, h=1e-5):
    return (f(x + h) - f(x)) / h

x = 2.0
print(numerical_derivative(f, x))

4.000010000027032


In [3]:
import numpy as np

def numer_deriv(f, x, h=0.001, method="center"):
    """
    {f(x+h) - f(x)} / h을 수치적으로 계산한다.
    f : 미분할 함수로 주어진 위치에서 함숫값 계산을 위해 사용 
    x : 미분계수를 구할 변수의 위치로
    일변수인 경우 int 또는 float
    다변수인 경우 넘파이 어레이 (d,) 벡터 
    h : 비율을 구할 작은 구간
    """
    if type(x) in (float, int):
        grad=[0,0]
        x_ = [x]
        var_type = 'scalar'
    else:
        grad = np.zeros(x.shape)
        x_ = x.copy().astype('float32')
        var_type = 'vector'

    for i, xi in enumerate(x_):
        original_value = x_[i]
        if method=='forward' :
            x_[i] = original_value + h
        else :
            x_[i] = original_value + (h/2)

        if var_type == 'scalar':
            gradplus = f(x_[i])
        else :
            gradplus = f(x_)

        if method=='forward' :
            x_[i] = original_value
        else:
            x_[i] = original_value - (h/2)
        
        if var_type == 'scalar' :
            gradminus = f(x_[i])
        else :
            gradminus = f(x_)

        grad[i] = (gradplus - gradminus) / h 
        x_[i] = original_value

    if var_type == 'scalar' : #❽
        return grad[0] 
    else :
        return grad






직접미분

In [4]:
import sympy as sp

# 변수 x를 정의
x = sp.symbols('x')

# 함수 g(x) = 3x^3 - 5x + 4 정의
g = 3*x**3 - 5*x + 4

# 함수 g(x)를 x에 대해 미분
g_prime = sp.diff(g, x)

# 결과 출력
print(f"g(x) = {g}")
print(f"g'(x) = {g_prime}")

g(x) = 3*x**3 - 5*x + 4
g'(x) = 9*x**2 - 5


자동미분

- 연쇄법칙을 함수에 반복적으로 적용 미분계수 자동으로 계산하는 방식

### 파이토치로 미분하기

In [5]:
import numpy as np
import torch
np.random.seed(0) # ❶
x = np.random.rand(6).reshape(2,3) #❷
x_tensor = torch.tensor(x) #❸ 
x_from_numpy = torch.from_numpy(x) 
x_Tensor = torch.Tensor(x)
x_as_Tensor = torch.as_tensor(x)

print(x, x.dtype)
print(x_tensor, x_tensor.dtype, x_tensor.requires_grad)
print(x_from_numpy, x_from_numpy.dtype, x_from_numpy.requires_grad)
print(x_Tensor, x_Tensor.dtype, x_Tensor.requires_grad)
print(x_as_Tensor, x_as_Tensor.dtype, x_as_Tensor.requires_grad)
x[0,0] = 100
print(x,x.dtype)

[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]] float64
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]]) torch.float32 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
[[100.           0.71518937   0.60276338]
 [  0.54488318   0.4236548    0.64589411]] float64


In [6]:
x_tensor_grad = torch.tensor(x,requires_grad=True)
print(x_tensor_grad, x_tensor_grad.dtype, x_tensor_grad.requires_grad)

tensor([[100.0000,   0.7152,   0.6028],
        [  0.5449,   0.4237,   0.6459]], dtype=torch.float64,
       requires_grad=True) torch.float64 True


In [7]:
def f_xy(x):
    return (x[0]*x[0] + 2*x[0])*np.log(x[1])

numer_deriv(f_xy, np.array([1, 2]), method="center")


array([2.77255303, 1.49989128])

In [11]:
f_xy  = lambda x : (x[0]**2 + 2*x[0])*np.log(x[1])
numer_deriv(f_xy, np.array([1, 2]))

array([2.77255299, 1.49989128])

자동미분


In [14]:
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad=True)
f_xy = (x**2 + 2*x) * torch.log(y)

torch.autograd.backward(f_xy, retain_graph=True)
print(x.grad)
print(y.grad)

df = torch.autograd.grad(f_xy, (x,y), retain_graph=True)
print(df)

tensor([2.7726])
tensor([1.5000])
(tensor([2.7726]), tensor([1.5000]))


In [15]:
x = torch.tensor([1.0], requires_grad=True)
f = (x**2 + 2*x) * torch.log(x)

print(x)
print(f)
print(x.grad)

print(x.grad_fn)
print(f.grad_fn)

tensor([1.], requires_grad=True)
tensor([0.], grad_fn=<MulBackward0>)
None
None


torch.autograd.grad사용

In [16]:
torch.autograd.backward(f, grad_tensors=torch.tensor([1.]), retain_graph=True)
print(x.grad)

tensor([3.])


In [17]:
df = torch.autograd.grad(f, x, retain_graph=True)
print(df)

(tensor([3.]),)


In [61]:
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad=True)
f_xy = (x**2 + 2*x) * torch.log(y)

torch.autograd.backward(f_xy, retain_graph=True)
print(x.grad)
print(y.grad)

df = torch.autograd.grad(f_xy, (x,y), retain_graph=True)

tensor([2.7726])
tensor([1.5000])


f_xy = x^3+4x*exp(x)
x로 미분 y로 직접미분

x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad=True)

In [1]:
import sympy  # sympy 라이브러리를 import 합니다. sympy는 기호적 계산을 위한 Python 라이브러리입니다.

x = sympy.Symbol('x')  # x를 기호 변수로 선언합니다.
y = sympy.Symbol('y')  # y를 기호 변수로 선언합니다.
z = sympy.Symbol('z')

f_xy_sympy = (x**3 + x*4)*sympy.exp(y)  # 주어진 함수 f(x, y) = (x^3 + 4x)e^y를 정의합니다.
f_xyz = (x**3 + x*4)*sympy.exp(y)*sympy.sin(z)
df_xy_x = sympy.diff(f_xy_sympy, x)  # f에 대해 x로 편미분합니다.
df_xy_y = sympy.diff(f_xy_sympy, y)  # f에 대해 y로 편미분합니다.


df_xyz_x = sympy.diff(f_xyz, x)
df_xyz_y = sympy.diff(f_xyz, y)
df_xyz_z = sympy.diff(f_xyz, z)

print(df_xy_x)  # x에 대한 편미분 결과를 출력합니다.
print(df_xy_y)  # y에 대한 편미분 결과를 출력합니다.

# x=4.0, y=5.0을 대입한 x에 대한 편미분 값과 y에 대한 편미분 값을 계산하여 출력합니다.
# .evalf(subs={x:4.0, y:5.0})은 편미분 결과에 x=4.0, y=5.0을 대입하여 수치적으로 평가하는 메소드입니다.
# "{:.4f}".format()은 계산된 결과를 소수점 넷째 자리까지 포맷팅하여 출력합니다.
print("{:.4f}".format(df_xy_x.evalf(subs={x:4.0, y:5.0})))
print("{:.4f}".format(df_xy_y.evalf(subs={x:4.0, y:5.0})))

print("{:.4f}".format(df_xyz_x.evalf(subs={x:4.0, y:5.0 ,z:1.0})))
print("{:.4f}".format(df_xyz_y.evalf(subs={x:4.0, y:5.0 ,z:1.0})))
print("{:.4f}".format(df_xyz_z.evalf(subs={x:4.0, y:5.0 ,z:1.0})))

2*x 4*y
2*x
4*y
8.0000
20.0000
6494.0391
9990.8294
6415.0378


In [50]:
import numpy as np

def numer_deriv(f, x, h=0.001, method="center"):
    """
    {f(x+h) - f(x)} / h을 수치적으로 계산한다.
    f : 미분할 함수로 주어진 위치에서 함숫값 계산을 위해 사용 
    x : 미분계수를 구할 변수의 위치로
    일변수인 경우 int 또는 float
    다변수인 경우 넘파이 어레이 (d,) 벡터 
    h : 비율을 구할 작은 구간
    """
    if type(x) in (float, int):
        grad=[0,0]
        x_ = [x]
        var_type = 'scalar'
    else:
        grad = np.zeros(x.shape)
        x_ = x.copy().astype('float32')
        var_type = 'vector'

    for i, xi in enumerate(x_):
        original_value = x_[i]
        if method=='forward' :
            x_[i] = original_value + h
        else :
            x_[i] = original_value + (h/2)

        if var_type == 'scalar':
            gradplus = f(x_[i])
        else :
            gradplus = f(x_)

        if method=='forward' :
            x_[i] = original_value
        else:
            x_[i] = original_value - (h/2)
        
        if var_type == 'scalar' :
            gradminus = f(x_[i])
        else :
            gradminus = f(x_)

        grad[i] = (gradplus - gradminus) / h 
        x_[i] = original_value

    if var_type == 'scalar' : #❽
        return grad[0] 
    else :
        return grad
    
    


In [53]:
def numer_deriv2(f, x, h=0.001, method="center"):
    """
    {f(x+h) - f(x)} / h을 수치적으로 계산한다.
    
    f      : 미분할 함수로 주어진 위치에서 함수값 계산을 위해 사용
    x      : 미분계수를 구할 변수의 위치로 
             일변수인 경우 int 또는 float 
             다변수인 경우 넘파이 어레이 (d,) 벡터
    h      : 비율을 구할 작은 구간
    """
    if type(x) in (float, int):    # ---- [1]
        grad = [0.0]
        x_ = [x]
        var_type = 'scalar'
    else:
        grad = np.zeros(x.shape)    # ---- [2]
        x_ = x.copy().astype('float32')
        var_type = 'vector'
    
    for i, xi in enumerate(x_):    # ---- [3]
        original_value = x_[i]
        
        if method == 'forward':    # ---- [4]
            x_[i] = original_value + h
        else:
            x_[i] = original_value + (h / 2)
        
        if var_type == 'scalar':    # ---- [5]   
            gradplus = f(*x_)
        else:
            gradplus = f(*x_)
        
        if method == 'forward':    # ---- [6]   
            x_[i] = original_value
        else:
            x_[i] = original_value - (h / 2)
        
        if var_type == 'scalar':
            gradminus = f(*x_)
        else:
            gradminus = f(*x_)
        
        grad[i] = (gradplus - gradminus) / h    # ---- [7]
    
    if var_type == 'scalar':    # ---- [8]
        return grad[0]
    else:
        return grad

In [56]:

# 심볼 정의
x, y, z = sp.symbols('x y z')

# 주어진 함수 정의
f_sym = (x**3 + 4*x) * sp.exp(y) * sp.sin(z)

# sympy 함수를 numpy 함수로 변환
f_np = sp.lambdify((x, y, z), f_sym, 'numpy')

point = np.array([4.0, 5.0, 1.0])

# 수치 미분 계산
gradient = numer_deriv2(f_np, point)

print("Gradient at point (4.0, 5.0, 1.0):", gradient)


Gradient at point (4.0, 5.0, 1.0): [6495.11747766 9991.27508271 6409.91010514]


In [60]:
f = lambda x : (x[0]**3 + x[0]*4)*np.exp(x[1])*np.sin(x[2])

#f = lambda x,y,z : (x**3 + x*4)*np.exp(y)*np.sin(z)

print(numer_deriv(f, np.array([4,5,1]),h=0.0001))


[ 6502.8585877  10004.79455543  6411.66329297]


In [8]:
import torch
x = torch.tensor([3.0], requires_grad=True)
y = torch.tensor([7.0], requires_grad=True)
z = torch.tensor([2.0], requires_grad=True)


f_xyz =(x**3 + x*3)*torch.exp(y)*torch.sin(z)
#torch.autograd.backward(f_xyz, retain_graph=True)

df = torch.autograd.grad(f_xyz, (x,y,z), retain_graph=True)
print(df)

(tensor([29914.9727]), tensor([35897.9648]), tensor([-16428.9746]))


In [ ]:
import sympy  # sympy 라이브러리를 import 합니다. sympy는 기호적 계산을 위한 Python 라이브러리입니다.

x = sympy.Symbol('x')  # x를 기호 변수로 선언합니다.
y = sympy.Symbol('y')  # y를 기호 변수로 선언합니다.
z = sympy.Symbol('z')
f_xy_sympy = (x**3 + x*4)*sympy.exp(y)  # 주어진 함수 f(x, y) = (x^3 + 4x)e^y를 정의합니다.
f_xyz = (x**3 + x*4)*sympy.exp(y)*sympy.sin(z)
df_xy_x = sympy.diff(f_xy_sympy, x)  # f에 대해 x로 편미분합니다.
df_xy_y = sympy.diff(f_xy_sympy, y)  # f에 대해 y로 편미분합니다.


df_xyz_x = sympy.diff(f_xyz, x)
df_xyz_y = sympy.diff(f_xyz, y)
df_xyz_z = sympy.diff(f_xyz, z)

print(df_xy_x)  # x에 대한 편미분 결과를 출력합니다.
print(df_xy_y)  # y에 대한 편미분 결과를 출력합니다.

# x=4.0, y=5.0을 대입한 x에 대한 편미분 값과 y에 대한 편미분 값을 계산하여 출력합니다.
# .evalf(subs={x:4.0, y:5.0})은 편미분 결과에 x=4.0, y=5.0을 대입하여 수치적으로 평가하는 메소드입니다.
# "{:.4f}".format()은 계산된 결과를 소수점 넷째 자리까지 포맷팅하여 출력합니다.
print("{:.4f}".format(df_xy_x.evalf(subs={x:4.0, y:5.0})))
print("{:.4f}".format(df_xy_y.evalf(subs={x:4.0, y:5.0})))

print("{:.4f}".format(df_xyz_x.evalf(subs={x:4.0, y:5.0 ,z:1.0})))
print("{:.4f}".format(df_xyz_y.evalf(subs={x:4.0, y:5.0 ,z:1.0})))
print("{:.4f}".format(df_xyz_z.evalf(subs={x:4.0, y:5.0 ,z:1.0})))

(3*x**2 + 4)*exp(y)
(x**3 + 4*x)*exp(y)
7717.4843
11873.0527
6494.0391
9990.8294
6415.0378
